In [1]:
import json

# данные
alcohol_products = [
    {
        "name": "Chateau Margaux 2015",
        "type": "wine",
        "vintage": 2015,
        "region": "Bordeaux",
        "grape": ["Cabernet Sauvignon", "Merlot"],
        "rating": { "sommelier": 4.8, "users": [5, 5, 4, 5] },
        "price": 199.99,
        "country": "France",
        "available": True
    },
    {
        "name": "Guinness Draught",
        "type": "beer",
        "volume": 0.44,
        "abv": 4.2,
        "tags": ["stout", "dark"],
        "price": 2.49,
        "country": "Ireland",
        "available": True
    },
    {
        "name": "Johnnie Walker Black Label",
        "type": "whisky",
        "age": 12,
        "abv": 40,
        "price": 32.00,
        "country": "Scotland",
        "available": True
    },
    {
        "name": "Craft IPA 7 Hops",
        "type": "beer",
        "volume": 0.5,
        "abv": 6.5,
        "tags": ["ipa", "hoppy", "bitter"],
        "price": 3.99,
        "country": "USA",
        "available": False
    },
    {
        "name": "Barolo Riserva",
        "type": "wine",
        "vintage": 2016,
        "region": "Piedmont",
        "grape": ["Nebbiolo"],
        "rating": { "sommelier": 4.6 },
        "price": 49.99,
        "country": "Italy",
        "available": True
    }
]

# сохранение в JSON file
with open('products.json', 'w', encoding='utf-8') as f:
    json.dump(alcohol_products, f, ensure_ascii=False, indent=2)

print("JSON file успешно создан")

JSON file успешно создан


In [ ]:
# перенес файла products.json в контейнер
# !docker cp products.json mongo_db:/products.json

# распарсинг products.json в контейнере, чтобы сделать из него 5 документов, где каждый - это отдельный товар
# !docker exec mongo_db mongoimport --db alcomarket --collection products --file /products.json --jsonArray

In [4]:
from pymongo import MongoClient
from pprint import pprint
import json

# 1. Подключение к MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["alcomarket"]
products = db["products"]

In [5]:
# 2. Очистка коллекции перед загрузкой (для повторного запуска)
products.drop()

# 3. Загрузка данных из файла products.json
with open("products.json", "r") as f:
    data = json.load(f)
    products.insert_many(data)

print("\n📦 Все товары:")
for doc in products.find():
    print(doc)


📦 Все товары:
{'_id': ObjectId('685ba12f1acdbbe70d6aa855'), 'name': 'Chateau Margaux 2015', 'type': 'wine', 'vintage': 2015, 'region': 'Bordeaux', 'grape': ['Cabernet Sauvignon', 'Merlot'], 'rating': {'sommelier': 4.8, 'users': [5, 5, 4, 5]}, 'price': 199.99, 'country': 'France', 'available': True}
{'_id': ObjectId('685ba12f1acdbbe70d6aa856'), 'name': 'Guinness Draught', 'type': 'beer', 'volume': 0.44, 'abv': 4.2, 'tags': ['stout', 'dark'], 'price': 2.49, 'country': 'Ireland', 'available': True}
{'_id': ObjectId('685ba12f1acdbbe70d6aa857'), 'name': 'Johnnie Walker Black Label', 'type': 'whisky', 'age': 12, 'abv': 40, 'price': 32.0, 'country': 'Scotland', 'available': True}
{'_id': ObjectId('685ba12f1acdbbe70d6aa858'), 'name': 'Craft IPA 7 Hops', 'type': 'beer', 'volume': 0.5, 'abv': 6.5, 'tags': ['ipa', 'hoppy', 'bitter'], 'price': 3.99, 'country': 'USA', 'available': False}
{'_id': ObjectId('685ba12f1acdbbe70d6aa859'), 'name': 'Barolo Riserva', 'type': 'wine', 'vintage': 2016, 'regio

# Запросы

In [6]:
print("\n🍷 Вина с рейтингом сомелье > 4.5:")
for doc in products.find({"type": "wine", "rating.sommelier": {"$gt": 4.5}}):
    print(doc)


🍷 Вина с рейтингом сомелье > 4.5:
{'_id': ObjectId('685ba12f1acdbbe70d6aa855'), 'name': 'Chateau Margaux 2015', 'type': 'wine', 'vintage': 2015, 'region': 'Bordeaux', 'grape': ['Cabernet Sauvignon', 'Merlot'], 'rating': {'sommelier': 4.8, 'users': [5, 5, 4, 5]}, 'price': 199.99, 'country': 'France', 'available': True}
{'_id': ObjectId('685ba12f1acdbbe70d6aa859'), 'name': 'Barolo Riserva', 'type': 'wine', 'vintage': 2016, 'region': 'Piedmont', 'grape': ['Nebbiolo'], 'rating': {'sommelier': 4.6}, 'price': 49.99, 'country': 'Italy', 'available': True}


In [7]:
print("\n📑 Только имя и цена всех товаров:")
for doc in products.find({}, {"name": 1, "price": 1, "_id": 0}):
    print(doc)


📑 Только имя и цена всех товаров:
{'name': 'Chateau Margaux 2015', 'price': 199.99}
{'name': 'Guinness Draught', 'price': 2.49}
{'name': 'Johnnie Walker Black Label', 'price': 32.0}
{'name': 'Craft IPA 7 Hops', 'price': 3.99}
{'name': 'Barolo Riserva', 'price': 49.99}


In [8]:
print("\n🌍 Уникальные страны происхождения:")
pprint(products.distinct("country"))


🌍 Уникальные страны происхождения:
['France', 'Ireland', 'Italy', 'Scotland', 'USA']


In [9]:
print("\n📊 Средняя цена пива по странам:")
pipeline = [
    {"$match": {"type": "beer"}},
    {"$group": {"_id": "$country", "avgPrice": {"$avg": "$price"}}},
    {"$sort": {"avgPrice": -1}}
]
for doc in products.aggregate(pipeline):
    pprint(doc)


📊 Средняя цена пива по странам:
{'_id': 'USA', 'avgPrice': 3.99}
{'_id': 'Ireland', 'avgPrice': 2.49}


In [10]:
print("\n🔄 Обновим цену Guinness до 2.99:")
products.update_one({"name": "Guinness Draught"}, {"$set": {"price": 2.99}})
pprint(products.find_one({"name": "Guinness Draught"}))


🔄 Обновим цену Guinness до 2.99:
{'_id': ObjectId('685ba12f1acdbbe70d6aa856'),
 'abv': 4.2,
 'available': True,
 'country': 'Ireland',
 'name': 'Guinness Draught',
 'price': 2.99,
 'tags': ['stout', 'dark'],
 'type': 'beer',
 'volume': 0.44}


In [11]:
print("\n➕ Добавим поле stock = 100 ко всем товарам:")
products.update_many({}, {"$set": {"stock": 100}})
pprint(products.find_one())


➕ Добавим поле stock = 100 ко всем товарам:
{'_id': ObjectId('685ba12f1acdbbe70d6aa855'),
 'available': True,
 'country': 'France',
 'grape': ['Cabernet Sauvignon', 'Merlot'],
 'name': 'Chateau Margaux 2015',
 'price': 199.99,
 'rating': {'sommelier': 4.8, 'users': [5, 5, 4, 5]},
 'region': 'Bordeaux',
 'stock': 100,
 'type': 'wine',
 'vintage': 2015}


In [12]:
print("\n❌ Удалим товары, которые недоступны (available: false):")
result = products.delete_many({"available": False})
print(f"Удалено документов: {result.deleted_count}")


❌ Удалим товары, которые недоступны (available: false):
Удалено документов: 1


In [13]:
print("\n📦 Все товары после удаления:")
for doc in products.find():
    pprint(doc)

print("\n📚 Индексы коллекции:")
print(products.index_information())


📦 Все товары после удаления:
{'_id': ObjectId('685ba12f1acdbbe70d6aa855'),
 'available': True,
 'country': 'France',
 'grape': ['Cabernet Sauvignon', 'Merlot'],
 'name': 'Chateau Margaux 2015',
 'price': 199.99,
 'rating': {'sommelier': 4.8, 'users': [5, 5, 4, 5]},
 'region': 'Bordeaux',
 'stock': 100,
 'type': 'wine',
 'vintage': 2015}
{'_id': ObjectId('685ba12f1acdbbe70d6aa856'),
 'abv': 4.2,
 'available': True,
 'country': 'Ireland',
 'name': 'Guinness Draught',
 'price': 2.99,
 'stock': 100,
 'tags': ['stout', 'dark'],
 'type': 'beer',
 'volume': 0.44}
{'_id': ObjectId('685ba12f1acdbbe70d6aa857'),
 'abv': 40,
 'age': 12,
 'available': True,
 'country': 'Scotland',
 'name': 'Johnnie Walker Black Label',
 'price': 32.0,
 'stock': 100,
 'type': 'whisky'}
{'_id': ObjectId('685ba12f1acdbbe70d6aa859'),
 'available': True,
 'country': 'Italy',
 'grape': ['Nebbiolo'],
 'name': 'Barolo Riserva',
 'price': 49.99,
 'rating': {'sommelier': 4.6},
 'region': 'Piedmont',
 'stock': 100,
 'type': 

In [14]:
# Создание индекса
products.create_index("type")

print("\n🧾 Индексы после создания индекса по type:")
pprint(products.index_information())


🧾 Индексы после создания индекса по type:
{'_id_': {'key': [('_id', 1)], 'v': 2},
 'type_1': {'key': [('type', 1)], 'v': 2}}
